# Evaluation

In [3]:
import numpy as np
import pandas as pd

## Execute and Read Data

In [9]:
# read test set
test = pd.read_csv("test.csv")

In [10]:
# generate predict and test data
#test = test.head(1000)
test_train = pd.DataFrame()
test_results = pd.DataFrame()
groups = test.groupby('user')

for i, group in groups:
    fraction = round(len(group)*0.6)
    test_train = test_train.append(group.iloc[:fraction])
    test_results = test_results.append(group.iloc[fraction:])

test_train.to_csv('test_train.csv', index=False)

In [11]:
# run the script
%run recsys.py -i "test_train.csv" -n 50 -o "results.csv"

IndexError: index out of bounds: 0 <= 491253 <= 475804, 0 <= 491254 <= 475804, 491253 <= 491254

In [17]:
# read result and add to dict
results = pd.read_csv("results.csv") 
results_dict = {}
groups = results.groupby('user')
for i, group in groups:
    user_id = group['user'].iloc[0]
    track_ids = set(group['track'])
    results_dict[user_id] = track_ids

In [18]:
# add test_results to dict
test_results_dict = {}
groups = test_results.groupby('user')
for i, group in groups:
    user_id = group['user'].iloc[0]
    track_ids = set(group['track'])
    test_results_dict[user_id] = track_ids

## Compare and create precision and recall

In [19]:
def get_precision(result_query, result_given):
    if len(result_query) == 0:
        return 0
    else:
        return len(set(result_given).intersection(result_query))/len(result_query)
def get_recall(result_query, result_given):
    if len(result_given) == 0:
        return 0
    else:
        return len(set(result_given).intersection(result_query))/len(result_given)

precisions = {}
recalls = {}
for key, value in results_dict.items():
    precisions[key] = get_precision(test_results_dict[key], results_dict[key])
    recalls[key] = get_recall(test_results_dict[key], results_dict[key])
    
precision = np.mean(list(precisions.values()))
recall = np.mean(list(recalls.values()))
print(precision)
print(recall)

0.0306404297539
0.0832352941176
